In [15]:
# This aglo is "AdaBoost Regression" ====> Capstone Project for Classification.

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import os
import warnings
warnings.filterwarnings('ignore')

In [17]:
df = pd.read_csv('cardekho_imputated.csv')
df.head(3)

,Unnamed: 0,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.7,796,46.3,5,120000
1,1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.9,1197,82.0,5,550000
2,2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.0,1197,80.0,5,215000


In [18]:
df.drop('car_name', axis = 1, inplace = True)

In [19]:
df.drop('brand', axis = 1, inplace = True)

In [20]:
df.columns

Index(['Unnamed: 0', 'model', 'vehicle_age', 'km_driven', 'seller_type',
       'fuel_type', 'transmission_type', 'mileage', 'engine', 'max_power',
       'seats', 'selling_price'],
      dtype='object')

In [21]:
# Encoding
Categorical = df.select_dtypes(include = 'object')

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [22]:
for i in Categorical:
    df[i] = le.fit_transform(df[i])

df.head(3)

,Unnamed: 0,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,0,7,9,120000,1,4,1,19.7,796,46.3,5,120000
1,1,54,5,20000,1,4,1,18.9,1197,82.0,5,550000
2,2,118,11,60000,1,4,1,17.0,1197,80.0,5,215000


In [23]:
# zscore
from scipy.stats import zscore
z = np.abs(zscore(df))
df1 = df[(z<3).all(axis = 1)]
print(df.shape)
print(df1.shape)

(15411, 12)
(14253, 12)


In [24]:
df = df1.copy()

In [25]:
# Split
X = df.drop('selling_price', axis = 1)
y = df['selling_price']

In [26]:
# Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_sc = sc.fit_transform(X)

In [27]:
# model_selection
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_sc, y, test_size = 0.2, random_state = 42)
X_train.shape, X_test.shape

((11402, 11), (2851, 11))

In [29]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [30]:
# create a function to evaluate model
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_scor = r2_score(true, predicted)
    return mae, rmse, r2_scor

In [32]:
# Beginning Model Training
models = { "Linear Regression " : LinearRegression(),
          "K-Neighbors Regressor " : KNeighborsRegressor(),
         "Decision Tree " : DecisionTreeRegressor(),
         "Random Forest Regressor ": RandomForestRegressor(),
         "AdaBoost Regressor": AdaBoostRegressor()
         }
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train)
    
    # Make Predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate train and test dataset
    model_train_mae, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_test_mae, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)
    
    print(list(models.keys())[i])
    
    print('Model performance for Training set')
    print('- Root Mean Squared Error : {:.4f}'.format(model_train_rmse))
    print('- Mean Absolute Error : {:.4f}'.format(model_train_mae))
    print('- R2 Score : {:.4f}'.format(model_train_r2))
    
    print('='*50)
    
    print('Model Performance for Test Set')
    print('- Root Mean Squared Error : {:.4f}'.format(model_test_rmse))
    print('- Mean Absolute Error : {:.4f}'.format(model_test_mae))
    print('- R2 Score : {:.4f}'.format(model_test_r2))
    
    print('='*35)
    print('\n')

Linear Regression 
Model performance for Training set
- Root Mean Squared Error : 249938.0053
- Mean Absolute Error : 163688.3038
- R2 Score : 0.7351
Model Performance for Test Set
- Root Mean Squared Error : 233346.1102
- Mean Absolute Error : 156501.3430
- R2 Score : 0.7546


K-Neighbors Regressor 
Model performance for Training set
- Root Mean Squared Error : 131936.3944
- Mean Absolute Error : 76470.3298
- R2 Score : 0.9262
Model Performance for Test Set
- Root Mean Squared Error : 159335.5972
- Mean Absolute Error : 96130.2525
- R2 Score : 0.8856


Decision Tree 
Model performance for Training set
- Root Mean Squared Error : 0.0000
- Mean Absolute Error : 0.0000
- R2 Score : 1.0000
Model Performance for Test Set
- Root Mean Squared Error : 162660.2235
- Mean Absolute Error : 97337.4255
- R2 Score : 0.8807


Random Forest Regressor 
Model performance for Training set
- Root Mean Squared Error : 47403.6331
- Mean Absolute Error : 28002.0424
- R2 Score : 0.9905
Model Performance for 

In [37]:
# Initialize few parameters for Hyperparameter tuning
knn_params = {'n_neighbors' : [2,3,10,20,40,50]}
rf_params = {'max_depth' : [5,8,15,None,10],
            'max_features' : [5,7,'auto',8],
            'min_samples_split':[2,8,15,20],
            'n_estimators':[100,200,500,1000]}

ada_params = {'n_estimators':[50,60,70,80],
             'loss':['linear', 'square', 'exponential']}

In [38]:
# Models List For HyperParameter tuning
randomcv_models = [('KNN', KNeighborsRegressor(), knn_params),
                  ('RF', RandomForestRegressor(), rf_params),
                  ('Adaboost', AdaBoostRegressor(), ada_params)]

In [ ]:
# HyperParameters Tuning
from sklearn.model_selection import RandomizedSearchCV

model_param = {}
for name, model, params in randomcv_models:
    random = RandomizedSearchCV(estimator = model,
                               param_distributions = params,
                               n_iter = 100,
                               cv = 3,
                               verbose = 2,
                               n_jobs = -1)
    random.fit(X_train, y_train)
    model_param[name] = random.best_params_

for model_name in model_param:
    print(f'--------------- Best Params for {model_name} ----------------')
    print(model_param[model_name])

Fitting 3 folds for each of 6 candidates, totalling 18 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits


In [ ]:
# Retraining the models with best parameters
models = {
    'Random Forest Regressor': RandomForestRegressor(n_estimators=100,min_smaples_split=2,max_features='auto',max_depth=None,n_jobs=-1),
    'K-Neighbors Regressor': KNeighborsRegressor(n_neighbors=10, n_jobs=-1),
    'Adaboost': AdaBoostRegressor(n_estimators=60, loss = 'linear')
}

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model
    
    # Make Predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate train and test dataset
    model_train_mae, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_test_mae, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)
    
    print(list(models.keys())[i])
    
    print('Model performance for Training set')
    print('- Root Mean Squared Error : {:.4f}'.format(model_train_rmse))
    print('- Mean Absolute Error : {:.4f}'.format(model_train_mae))
    print('- R2 Score : {:.4f}'.format(model_train_r2))
    
    print('='*50)
    
    print('Model Performance for Test Set')
    print('- Root Mean Squared Error : {:.4f}'.format(model_test_rmse))
    print('- Mean Absolute Error : {:.4f}'.format(model_test_mae))
    print('- R2 Score : {:.4f}'.format(model_test_r2))
    
    print('='*35)
    print('\n')